<a href="https://colab.research.google.com/github/hrishipoola/Ebay_Kleinanzeigen_Scrape/blob/main/Ebay_Kleinanzeigen_Peppa_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ebay Kleinanzeigen Peppa Pig Scrape

## Table of Contents

1. Introduction
2. Install & Import Packages
3. BeautifulSoup
4. Scrapy
5. References 

## 1. Introduction

My daughter Danae's birthday is approaching. She loves Peppa Pig. I decided to scrape Ebay Kleinanzeigen (Germany's eBay classifieds) for inspiration and ideas that I could later organize by seller neighborhood and price. I was surprised by the variety and creativity of Peppa gear! 

With a search for 'Peppa' in Berlin, I scraped product title, image, price, neighborhood, zipcode, and link for each item using 2 approaches. 

First, I used BeautifulSoup and constructed urls based on search criteria for the first 8 pages (what I was allowed to scrape), looped through each page to scrape contents, and created a dataframe. I was able to scrape 125 items. 

Next, I used a more elegant approach building a [Scrapy Spider](https://docs.scrapy.org/en/latest/topics/spiders.html) that scraped each item for contents and, if there was a next page link, followed the next page link using a callback and continued to scrape until there were no more pages, finally creating a dataframe. I was able to scrape 174 items using this approach. 

In both, since I didn't use Selenium, in order to simulate natural queries, I cycled randomly through different user agents and sleep time intervals, an approach I learned from [Achim Tack](https://github.com/AchimTack/GoogleTrafficParser/blob/master/google_traffic_parser.py). I also found that for more challenging-to-parse text between unique elements, xpath worked better than css.  


## 2. Install & Import Packages

In [1]:
import pandas as pd 
import numpy as np
import io
import os

# Web scraping using BeautifulSoup and converting to pandas dataframe
import requests 
import urllib.request 
from urllib.request import urlopen
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re

!pip install scrapy
import scrapy
from scrapy import Selector
from scrapy.crawler import CrawlerProcess

from time import sleep
from warnings import warn
import random
from random import randint
from random import choice
import time
from time import sleep

# Merging dataframes in dictionary of dataframes
from functools import partial, reduce

import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display,HTML # Image to dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Mount google drive
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3. BeautifulSoup

In [3]:
start_url = 'https://www.ebay-kleinanzeigen.de'
url = 'https://www.ebay-kleinanzeigen.de/s-berlin/peppa/k0l3331'
search_item = 'peppa'
place = 'berlin'


agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36',
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0',
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15',
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
          'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0',
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
          'Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0']

names=[]; images=[]; prices=[]; neighborhoods=[]; zipcodes=[]; links=[] 


# Create urls 
start_page=1; end_page=8 # Only allows scrape through 8 pages
urls = ['https://www.ebay-kleinanzeigen.de/s-{}/seite:{}/{}/k0l3331'.format(place,i,search_item) for i in range(start_page,end_page+1)]
    
for url in urls:
    # cycles randomly through different user agents and time intervals to simulate more natural queries
    user_agent = choice(agents)
    headers = {'User-Agent': user_agent}
    sleeptime = float(randint(1,20))/5
    sleep(sleeptime)
    
    html = requests.get(url=url, headers=headers).content
    sel = Selector(text = html) # Create the Selector object sel from html. Selector returns a list

    name = sel.css('a.ellipsis::text').extract()
    name = [item.replace('- FIRLEFANZ', '') for item in name]

    image = sel.css('div.imagebox.srpimagebox::attr(data-imgsrc)').extract()
        
    price = sel.xpath('//*[@class="aditem-details"]/strong/text()').extract() 
    price = [re.sub('[^0-9]', '', item) for item in price] # replace all non numeric characters in each string

    neighborhood = sel.css('div[class=aditem-details]::text').extract()
    neighborhood = list(filter(None, [item.strip() for item in neighborhood])) # filter out none elements, strip text
    neighborhood = [item for item in neighborhood if not item.isdigit()]    
        
    zipcode = sel.css('div[class=aditem-details]::text').extract()
    zipcode = list(filter(None, [item.strip() for item in zipcode])) # filter out none elements, strip text
    zipcode = [item for item in zipcode if item.isdigit()] #keep only numeric strings
        
    link = sel.css('a.ellipsis::attr(href)').extract() # extract href attribute from element a of class ellipsis 
    link = [start_url + x for x in link]
        
    names.append(name)
    images.append(image)
    prices.append(price)
    neighborhoods.append(neighborhood)
    zipcodes.append(zipcode)
    links.append(link)
    print('{} completed.'.format(url))

# Flatten list of lists
names = [item for items in names for item in items]
images = [item for items in images for item in items]
prices = [item for items in prices for item in items]
neighborhoods = [item for items in neighborhoods for item in items]
zipcodes = [item for items in zipcodes for item in items]
links = [item for items in links for item in items] 

# Create dataframe
peppa = pd.DataFrame((zip(names, images, prices, neighborhoods, zipcodes, links)), columns=['Item','Image','Price (€)', 'Neighborhood', 'Zipcode', 'Link'])
peppa['Image'] = peppa['Image'].str.replace('(.*)', '<img src="\\1" style="max-height:124px;"></img>') 
peppa.to_csv('ebay_kleinanzeigen_scrape.csv', index=False)

display(HTML(peppa.to_html(escape=False)))

https://www.ebay-kleinanzeigen.de/s-berlin/seite:1/peppa/k0l3331 completed.
https://www.ebay-kleinanzeigen.de/s-berlin/seite:2/peppa/k0l3331 completed.
https://www.ebay-kleinanzeigen.de/s-berlin/seite:3/peppa/k0l3331 completed.
https://www.ebay-kleinanzeigen.de/s-berlin/seite:4/peppa/k0l3331 completed.
https://www.ebay-kleinanzeigen.de/s-berlin/seite:5/peppa/k0l3331 completed.
https://www.ebay-kleinanzeigen.de/s-berlin/seite:6/peppa/k0l3331 completed.
https://www.ebay-kleinanzeigen.de/s-berlin/seite:7/peppa/k0l3331 completed.
https://www.ebay-kleinanzeigen.de/s-berlin/seite:8/peppa/k0l3331 completed.


,Item,Image,Price (€),Neighborhood,Zipcode,Link
0,17×Jeggings Leggings Peppa Minnie Elsa Mädchen 110/116 H&M z.T,,40,Reinickendorf,13435,https://www.ebay-kleinanzeigen.de/s-anzeige/17-jeggings-leggings-peppa-minnie-elsa-maedchen-110-116-h-m-z-t/1669053066-22-3457
1,Peppa Pig Rucksack rosa Kindergarten,,10,Pankow,13189,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-pig-rucksack-rosa-kindergarten/1668720079-18-3472
2,Bekleidungspaket Set Mädchen 122/128 H&M Peppa Wutz und Barbies,,35,Hellersdorf,12619,https://www.ebay-kleinanzeigen.de/s-anzeige/bekleidungspaket-set-maedchen-122-128-h-m-peppa-wutz-und-barbies/1668612850-22-3511
3,Peppa Wutz Big Spielhaus,,25,Tempelhof,12305,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-wutz-big-spielhaus/1668589343-23-3410
4,Vtech peppa Wutz Uhr,,11,Tempelhof,12099,https://www.ebay-kleinanzeigen.de/s-anzeige/vtech-peppa-wutz-uhr/1668582703-23-3409
5,Peppa Pig First Look and Find Board book Buch Englisch eOne,,10,Mitte,10117,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-pig-first-look-and-find-board-book-buch-englisch-eone/1668360774-76-3520
6,Peppa pig kleid,,10,Neukölln,12057,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-pig-kleid/1668237029-22-9633
7,Peppa Pig Bettwäsche 135x200 50x70,,10,Neukölln,12057,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-pig-bettwaesche-135x200-50x70/1668211932-90-9633
8,Peppa Wutz Figuren,,5,Prenzlauer Berg,10407,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-wutz-figuren/1668206550-23-3493
9,10 Bücher für Kleinkinder Peppa fühlen Steiff Töpfchen Set,,10,Spandau,13589,https://www.ebay-kleinanzeigen.de/s-anzeige/10-buecher-fuer-kleinkinder-peppa-fuehlen-steiff-toepfchen-set/1668010355-76-3437


## 4. Scrapy

In [4]:
search_item = 'peppa'
place = 'berlin'

start_url = 'https://www.ebay-kleinanzeigen.de'
url = ('https://www.ebay-kleinanzeigen.de/s-{}/{}/k0l3331'.format(place,search_item))

# To get user agent for your browser, go to http://httpbin.org/get
agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36',
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0',
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15',
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
          'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0',
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
          'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
          'Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0']

user_agent = choice(agents) #user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:78.0) Gecko/20100101 Firefox/78.0'
headers = {'User-Agent': user_agent}
html = requests.get(url=url, headers=headers).content
sel = Selector(text = html)

sleeptime = float(randint(1,20))/5

names=[]; images=[]; prices=[]; neighborhoods=[]; zipcodes=[]; links=[] 

class Ebay_Spider(scrapy.Spider): 
    name = 'ebay_spider'
    allowed_domans = ['https://www.ebay-kleinanzeigen.de']
    
    def start_requests(self):
        yield scrapy.Request(url = url, headers = headers, callback = self.parse_ebay)
  
    def parse_ebay(self, response): 
        name = response.css('a.ellipsis::text').extract()
        name = [item.replace('- FIRLEFANZ', '') for item in name]

        image = response.css('div.imagebox.srpimagebox::attr(data-imgsrc)').extract()
        
        price = response.xpath('//*[@class="aditem-details"]/strong/text()').extract() 
        price = [re.sub('[^0-9]', '', item) for item in price] # replace all non numeric characters in each string

        neighborhood = response.css('div[class=aditem-details]::text').extract()
        neighborhood = list(filter(None, [item.strip() for item in neighborhood])) # filter out none elements, strip text
        neighborhood = [item for item in neighborhood if not item.isdigit()]
        
        
        zipcode = response.css('div[class=aditem-details]::text').extract()
        zipcode = list(filter(None, [item.strip() for item in zipcode])) # filter out none elements, strip text
        zipcode = [item for item in zipcode if item.isdigit()] #keep only numeric strings
        
        link = response.css('a.ellipsis::attr(href)').extract() # extract href attribute from element a of class ellipsis 
        link = [start_url + x for x in link]
        
        names.append(name)
        images.append(image)
        prices.append(price)
        neighborhoods.append(neighborhood)
        zipcodes.append(zipcode)
        links.append(link)
        
        next_page = response.css('.pagination-next::attr(href)').extract_first() # Make sure to use extract_first() and not extract() so you get url and not list of urls
        next_page = start_url + next_page
        print(next_page)
        sleep(sleeptime)
        if next_page is not None:
            yield response.follow(url = next_page, headers = headers, callback = self.parse_ebay)
            
process = CrawlerProcess()
process.crawl(Ebay_Spider)
process.start()

# Flatten list of lists
names = [item for items in names for item in items]
images = [item for items in images for item in items]
prices = [item for items in prices for item in items]
neighborhoods = [item for items in neighborhoods for item in items]
zipcodes = [item for items in zipcodes for item in items]
links = [item for items in links for item in items] 

# Create dataframe
peppa = pd.DataFrame((zip(names, images, prices, neighborhoods, zipcodes, links)), columns=['Item','Image','Price (€)', 'Neighborhood', 'Zipcode', 'Link'])
peppa['Image'] = peppa['Image'].str.replace('(.*)', '<img src="\\1" style="max-height:124px;"></img>') 
peppa.to_csv('ebay_kleinanzeigen_spider_scrape.csv', index=False)

display(HTML(peppa.to_html(escape=False)))


2021-02-16 09:09:02 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-02-16 09:09:02 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Oct  8 2020, 12:12:24) - [GCC 8.4.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.4.5, Platform Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
2021-02-16 09:09:02 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-02-16 09:09:02 [scrapy.crawler] INFO: Overridden settings:
{}
2021-02-16 09:09:02 [scrapy.extensions.telnet] INFO: Telnet Password: 695bedf1879d2fe1
2021-02-16 09:09:02 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-02-16 09:09:02 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.download

https://www.ebay-kleinanzeigen.de/s-berlin/seite:2/peppa/k0l3331


2021-02-16 09:09:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ebay-kleinanzeigen.de/s-berlin/seite:2/peppa/k0l3331> (referer: https://www.ebay-kleinanzeigen.de/s-berlin/peppa/k0l3331)


https://www.ebay-kleinanzeigen.de/s-berlin/seite:3/peppa/k0l3331


2021-02-16 09:09:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ebay-kleinanzeigen.de/s-berlin/seite:3/peppa/k0l3331> (referer: https://www.ebay-kleinanzeigen.de/s-berlin/seite:2/peppa/k0l3331)


...

https://www.ebay-kleinanzeigen.de/s-berlin/seite:7/peppa/k0l3331


2021-02-16 09:09:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ebay-kleinanzeigen.de/s-berlin/seite:7/peppa/k0l3331> (referer: https://www.ebay-kleinanzeigen.de/s-berlin/seite:6/peppa/k0l3331)


https://www.ebay-kleinanzeigen.de/s-berlin/seite:8/peppa/k0l3331


2021-02-16 09:09:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ebay-kleinanzeigen.de/s-berlin/seite:8/peppa/k0l3331> (referer: https://www.ebay-kleinanzeigen.de/s-berlin/seite:7/peppa/k0l3331)
2021-02-16 09:09:11 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.ebay-kleinanzeigen.de/s-berlin/seite:8/peppa/k0l3331> (referer: https://www.ebay-kleinanzeigen.de/s-berlin/seite:7/peppa/k0l3331)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/scrapy/utils/defer.py", line 120, in iter_errback
    yield next(it)
  File "/usr/local/lib/python3.6/dist-packages/scrapy/utils/python.py", line 353, in __next__
    return next(self.data)
  File "/usr/local/lib/python3.6/dist-packages/scrapy/utils/python.py", line 353, in __next__
    return next(self.data)
  File "/usr/local/lib/python3.6/dist-packages/scrapy/core/spidermw.py", line 62, in _evaluate_iterable
    for r in iterable:
  File "/usr/local/lib/python3.6/dist-packages/scra

,Item,Image,Price (€),Neighborhood,Zipcode,Link
0,17×Jeggings Leggings Peppa Minnie Elsa Mädchen 110/116 H&M z.T,,40,Reinickendorf,13435,https://www.ebay-kleinanzeigen.de/s-anzeige/17-jeggings-leggings-peppa-minnie-elsa-maedchen-110-116-h-m-z-t/1669053066-22-3457
1,Peppa Pig Rucksack rosa Kindergarten,,10,Pankow,13189,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-pig-rucksack-rosa-kindergarten/1668720079-18-3472
2,Bekleidungspaket Set Mädchen 122/128 H&M Peppa Wutz und Barbies,,35,Hellersdorf,12619,https://www.ebay-kleinanzeigen.de/s-anzeige/bekleidungspaket-set-maedchen-122-128-h-m-peppa-wutz-und-barbies/1668612850-22-3511
3,Peppa Wutz Big Spielhaus,,25,Tempelhof,12305,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-wutz-big-spielhaus/1668589343-23-3410
4,Vtech peppa Wutz Uhr,,11,Tempelhof,12099,https://www.ebay-kleinanzeigen.de/s-anzeige/vtech-peppa-wutz-uhr/1668582703-23-3409
5,Peppa Pig First Look and Find Board book Buch Englisch eOne,,10,Mitte,10117,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-pig-first-look-and-find-board-book-buch-englisch-eone/1668360774-76-3520
6,Peppa pig kleid,,10,Neukölln,12057,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-pig-kleid/1668237029-22-9633
7,Peppa Pig Bettwäsche 135x200 50x70,,10,Neukölln,12057,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-pig-bettwaesche-135x200-50x70/1668211932-90-9633
8,Peppa Wutz Figuren,,5,Prenzlauer Berg,10407,https://www.ebay-kleinanzeigen.de/s-anzeige/peppa-wutz-figuren/1668206550-23-3493
9,10 Bücher für Kleinkinder Peppa fühlen Steiff Töpfchen Set,,10,Spandau,13589,https://www.ebay-kleinanzeigen.de/s-anzeige/10-buecher-fuer-kleinkinder-peppa-fuehlen-steiff-toepfchen-set/1668010355-76-3437


## 5. References

https://docs.scrapy.org/en/latest/topics/spiders.html

https://docs.scrapy.org/en/latest/intro/tutorial.html#a-shortcut-for-creating-requests

https://stackoverflow.com/questions/52280630/scrapy-with-multiple-pages

https://resbazsql.github.io/lc-webscraping/05-scraping-multiple-pages-with-scrapy/

https://stackoverflow.com/questions/61488724/how-to-set-scrapy-spider-to-run-over-multiple-pages-without-next-page-button

https://stackoverflow.com/questions/52512750/how-to-stop-scrapy-from-crawling-when-it-reaches-the-starting-point-of-previous

https://gist.github.com/fadere/fe9d471fb605d7975ee3

https://github.com/tax0r/Ebay-Kleinanzeigen-Scraper/blob/master/Ebay-Kleinanzeigen%20Scraper/main.py
 
https://letslearnabout.net/tutorial/scrapy-tutorial/python-scrapy-tutorial-for-beginners-01-creating-your-first-spider/

https://github.com/HicBoux/Ebay-kleinanzeigen-scrapy-elastic/blob/master/ebaykleinanzeigen/ebaykleinanzeigen/spiders/ebay_kleinanzeigen.py

https://omnianalytics.io/2018/08/28/ebay-web-scrape-tutorial/

https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe/53469293

https://datascience.stackexchange.com/questions/38083/display-images-url-inside-pandas-dataframe

https://stackoverflow.com/questions/53171113/how-to-extract-text-which-lies-after-strong-tag-in-element

https://www.youtube.com/watch?v=7V2C0e66ZDk

https://github.com/AchimTack/GoogleTrafficParser/blob/master/google_traffic_parser.py
